In [1]:
!nvidia-smi

Wed Jul 31 22:41:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:01:00.0 Off |                   On |
| N/A   29C    P0              26W / 165W |     50MiB / 24576MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [2]:
from dataclasses import dataclass
from typing import Any, Union, Tuple, Callable, Optional
from functools import partial
import time

import jax
import jax.numpy as jnp
import flax
import flax.linen as nn
from flax.training.train_state import TrainState
import optax

from qdax import environments, environments_v1
from jax import random
import wandb

import pickle
from optax import exponential_decay
from IPython.display import HTML
from brax.io import html
import os
import jax.debug
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import optax 
from flax.linen.initializers import constant, orthogonal, lecun_uniform
from typing import Sequence, NamedTuple, Any
from flax.training.train_state import TrainState
import distrax
from wrappers import (
    LogWrapper,
    BraxGymnaxWrapper,
    VecEnv,
    NormalizeVecObservation,
    NormalizeVecReward,
    ClipAction,
)

In [4]:
class ActorCritic(nn.Module):
    action_dim: Sequence[int]
    activation: str = "tanh"
    
    @nn.compact
    def __call__(self, x):
        if self.activation == "relu":
            activation = nn.relu
        else:
            activation = nn.tanh
            
        actor_mean = nn. Dense(
            64, kernel_init = orthogonal(np.sqrt(2)), bias_init = constant(0.0)
        )(x)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            64, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(actor_mean)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            self.action_dim, kernel_init=orthogonal(0.01), bias_init=constant(0.0)
        )(actor_mean)
        actor_logstd = self.param("log_std", lambda _, shape: jnp.log(0.5)*jnp.ones(shape), (self.action_dim,))
        pi = distrax.MultivariateNormalDiag(loc=actor_mean, scale_diag=jnp.exp(actor_logstd))
        
        critic = nn.Dense(
            64, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(x)
        critic = activation(critic)
        critic = nn.Dense(
            64, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(critic)
        critic = activation(critic)
        critic = nn.Dense(1, kernel_init=orthogonal(1.0), bias_init=constant(0.0))(critic)
        
        return pi, jnp.squeeze(critic, axis=-1)
        
        

In [6]:
class Transition(NamedTuple):
    done: jnp.ndarray
    action: jnp.ndarray
    value: jnp.ndarray
    reward: jnp.ndarray
    log_prob: jnp.ndarray
    obs: jnp.ndarray
    info: jnp.ndarray
    
def make_train(config):
    config["NUM_UPDATES"] = (
        config["TOTAL_TIMESTEPS"] // config["NUM_STEPS"] // config["NUM_ENVS"]
    )
    config["MINIBATCH_SIZE"] = (
        config["NUM_ENVS"] * config["NUM_STEPS"] // config["NUM_MINIBATCHES"]
    )
    env, env_params = BraxGymnaxWrapper(config["ENV_NAME"]), None
    env = LogWrapper(env)
    env = ClipAction(env)
    env = VecEnv(env)
    if config["NORMALIZE_ENV"]:
        env = NormalizeVecObservation(env)
        env = NormalizeVecReward(env, config["GAMMA"])
    
    def linear_schedule(count):
        frac = (
            1.0
            - (count // (config["NUM_MINIBATCHES"] * config["UPDATE_EPOCHS"]))
            / config["NUM_UPDATES"]
        )
        return config["LR"] * frac
    
    def train(rng):
        # INIT NETWORK
        network = ActorCritic(
            env.action_space(env_params).shape[0], activation=config["ACTIVATION"]
        )
        rng, _rng = jax.random.split(rng)
        init_x = jnp.zeros(env.observation_space(env_params).shape)
        network_params = network.init(_rng, init_x)
        if config["ANNEAL_LR"]:
            tx = optax.chain(
                optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
                optax.adam(
                    learning_rate=linear_schedule,
                    eps=1e-5,
                ),
            )
            
        else:
            tx = optax.chain(
                optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
                optax.adam(
                    learning_rate=config["LR"],
                    eps=1e-5,
                ),
            )
            
        train_state = TrainState.create(
            apply_fn=network.apply,
            params=network_params,
            tx=tx,
        )
        
        # INIT ENV
        rng, _rng = jax.random.split(rng)
        reset_rng = jax.random.split(_rng, config["NUM_ENVS"])
        obsv, env_state = env.reset(reset_rng, env_params)
        
        # TRAIN LOOP
        def _update_step(runner_state, unused):
            # COLLECT TRAJECTORIES
            def _env_step(runner_state, unused):
                train_state, env_state, last_obs, rng = runner_state
                
                # SELECT ACTION
                rng, _rng = jax.random.split(rng)
                pi, value = network.apply(train_state.params, last_obs)
                action = pi.sample(seed=_rng)
                log_prob = pi.log_prob(action)
                
                # STEP ENV
                rng, _rng = jax.random.split(rng)
                rng_step = jax.random.split(_rng, config["NUM_ENVS"])
                obsv, env_state, reward, done, info = env.step(
                    rng_step, env_state, action, env_params,
                )    
                transition = Transition(
                    done, action, value, reward, log_prob, last_obs, info
                )
                runner_state = (train_state, env_state, obsv, rng)
                return runner_state, transition
            
            runner_state, traj_batch = jax.lax.scan(
                _env_step, 
                runner_state, 
                None, 
                length=config["NUM_STEPS"]
            )
            
            # COMPUTE ADVANTAGES
            train_state, env_state, last_obs, rng = runner_state
            _, last_val = network.apply(train_state.params, last_obs)
            
            def _calculate_gae(traj_batch, last_val):
                def _get_advantages(gae_and_next_value, transition):
                    gae, next_value = gae_and_next_value
                    done, value, reward = (
                        transition.done,
                        transition.value,
                        transition.reward,
                    )
                    delta = reward + config["GAMMA"] * next_value * (1 - done) - value
                    gae = (
                        delta
                        + config["GAMMA"] * config["GAE_LAMBDA"] * (1 - done) * gae
                    )
                    return (gae, value), gae
                
                _, advantages = jax.lax.scan(
                    _get_advantages,
                    (jnp.zeros_like(last_val), last_val),
                    traj_batch,
                    reverse=True,
                    unroll=16,
                )
                
                return advantages, advantages + traj_batch.value
            
            advantages, targets = _calculate_gae(traj_batch, last_val)
            
            # UPDATE NETWORK
            def _update_epoch(update_state, unused):
                def _update_minibatch(train_state, batch_info):
                    traj_batch, advantages, targets = batch_info
                    
                    def _loss_fn(params, traj_batch, gae, targets):
                        # RERUN NETWORK
                        pi, value = network.apply(params, traj_batch.obs)
                        log_prob = pi.log_prob(traj_batch.action)
                        
                        # CALCULATE VALUE LOSS
                        value_pred_clipped = traj_batch.value + (
                            value - traj_batch.value
                        ).clip(-config["CLIP_EPS"], config["CLIP_EPS"])
                        value_losses = jnp.square(value - targets)
                        value_losses_clipped = jnp.square(value_pred_clipped - targets)
                        value_loss = 0.5 * jnp.mean(jnp.maximum(value_losses, value_losses_clipped))
                        
                        # CALCULATE ACTOR LOSS
                        ratio = jnp.exp(log_prob - traj_batch.log_prob)
                        gae = (gae - gae.mean()) / (gae.std() + 1e-8)
                        loss_actor1 = ratio * gae
                        loss_actor2 = (
                            jnp.clip(
                            ratio, 
                            1.0 - config["CLIP_EPS"], 
                            1.0 + config["CLIP_EPS"]
                            ) 
                            * gae
                        )
                        loss_actor = -jnp.mean(jnp.minimum(loss_actor1, loss_actor2))
                        entropy = jnp.mean(pi.entropy())
                        
                        total_loss = (
                            loss_actor
                            + config["VF_COEF"] * value_loss
                            - config["ENT_COEF"] * entropy
                        )
                        return total_loss, (value_loss, loss_actor, entropy)
                    
                    grad_fn = jax.value_and_grad(_loss_fn, has_aux=True)
                    total_loss, grad = grad_fn(
                        train_state.params, 
                        traj_batch, 
                        advantages, 
                        targets)
                    train_state = train_state.apply_gradients(grads=grad)
                    return train_state, total_loss
                
                train_state, traj_batch, advantages, targets, rng = update_state
                rng, _rng = jax.random.split(rng)
                batch_size = config["MINIBATCH_SIZE"] * config["NUM_MINIBATCHES"]
                assert (
                    batch_size == config["NUM_ENVS"] * config["NUM_STEPS"]
                ), "batch size must be equal to number of steps * number of envs"
                
                permutation = jax.random.permutation(_rng, batch_size)
                batch = (traj_batch, advantages, targets)
                batch = jax.tree_util.tree_map(
                    lambda x: x.reshape((batch_size,) + x.shape[2:]), batch
                )
                shuffled_batch = jax.tree_util.tree_map(
                    lambda x: jnp.take(x, permutation, axis=0), batch   
                )
                minibatches = jax.tree_util.tree_map(
                    lambda x: jnp.reshape(
                        x, [config["NUM_MINIBATCHES"], -1] + list(x.shape[1:])
                    ),
                    shuffled_batch,
                )
                
                train_state, total_loss = jax.lax.scan(
                    _update_minibatch,
                    train_state,
                    minibatches,
                )
                update_state = (train_state, traj_batch, advantages, targets, rng)
                return update_state, total_loss
            
            update_state = (train_state, traj_batch, advantages, targets, rng)
            update_state, loss_info = jax.lax.scan(
                _update_epoch,
                update_state,
                None,
                length=config["UPDATE_EPOCHS"], 
            )
            train_state = update_state[0]
            metric = traj_batch.info
            rng = update_state[-1]
            if config.get("DEBUG"):
                
                def callback(info):
                    return_values = info["returned_episode_returns"][
                        info["returned_episode"]
                    ]
                    timesteps = (
                        info["timestep"][info["returned_episode"]] * config["NUM_ENVS"]
                    )
                    for t in range(len(timesteps)):
                        print(
                            f"global step={timesteps[t]}, episodic return={return_values[t]}"
                        )
                        
                jax.debug.callback(callback, metric)
                
            runner_state = (train_state, env_state, last_obs, rng)
            return runner_state, metric
        
        rng, _rng = jax.random.split(rng)
        runner_state = (train_state, env_state, obsv, _rng)
        runner_state, metric = jax.lax.scan(
            _update_step,
            runner_state,
            None,
            length=config["NUM_UPDATES"],
        )
        return {"runner_state": runner_state, "metrics": metric, "net": network, "env": env}
    
    return train        

In [7]:
config = {
    "LR": 1e-3,
    "NUM_ENVS": 2048,
    "NUM_STEPS": 10,
    "TOTAL_TIMESTEPS": 1e7,
    "UPDATE_EPOCHS": 4,
    "NUM_MINIBATCHES": 32,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_EPS": 0.2,
    "ENT_COEF": 0.0,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "ACTIVATION": "tanh",
    "ENV_NAME": "walker2d",
    "ANNEAL_LR": False,
    "NORMALIZE_ENV": True,
    "DEBUG": True,
}
rng = jax.random.PRNGKey(30)
#train_jit = jax.jit(make_train(config))
#out = train_jit(rng)
train = make_train(config)
out = train(rng)

global step=253952, episodic return=235.95838928222656
global step=256000, episodic return=233.56362915039062
global step=256000, episodic return=237.2659454345703
global step=260096, episodic return=238.5930938720703
global step=264192, episodic return=242.6988525390625
global step=266240, episodic return=246.70989990234375
global step=266240, episodic return=246.55409240722656
global step=266240, episodic return=242.85069274902344
global step=268288, episodic return=244.53115844726562
global step=268288, episodic return=244.44581604003906
global step=268288, episodic return=248.85414123535156
global step=268288, episodic return=244.4706573486328
global step=268288, episodic return=245.4765167236328
global step=272384, episodic return=247.1748504638672
global step=272384, episodic return=248.83609008789062
global step=272384, episodic return=247.71717834472656
global step=274432, episodic return=251.81187438964844
global step=274432, episodic return=244.86068725585938
global step=2764

In [8]:
import brax
from brax.io import html

In [22]:
def get_env(env_name):
    if env_name == "hopper_uni":
        episode_length = 1000
        
        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)
    elif env_name == "halfcheetah_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)
        
    elif env_name == "walker2d_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)	
    elif env_name == "ant_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, use_contact_forces=True, exclude_current_positions_from_observation=True)
    elif env_name == "humanoid_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)	
    '''
    elif env_name == "ant_omni":
        episode_length = 250
        max_bd = 30.

        env = environments.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=False)	
    elif env_name == "humanoid_uni":
        episode_length = 1000
        max_bd = 1.

        env = environments.create(env_name, episode_length=episode_length)	
    else:
        ValueError(f"Environment {env_name} not supported.")
    '''
    return env

In [34]:
env = BraxGymnaxWrapper(config["ENV_NAME"])
#env = BraxGymnaxWrapper('ant')
#env = LogWrapper(env)
env = ClipAction(env)
#env = VecEnv(env)
if config["NORMALIZE_ENV"]:
    env = NormalizeVecObservation(env)
    #env = NormalizeVecReward(env, config["GAMMA"])

In [35]:
#env = out["env"]
net = out["net"]
params = out["runner_state"][0].params

In [36]:
#env = get_env("hopper_uni")
reset_fn = jax.jit(env.reset)
step_fn = jax.jit(env.step)

In [39]:
rng = jax.random.PRNGKey(30)
obs, state = reset_fn(rng)
rollout = [state]
total_reward = 0
steps = 0
s = 0
while True:
    s += 1
    steps += 1
    print(f"Step: {steps}")
    rng, _rng, __rng = jax.random.split(rng, 3)
    #pi, _ = net.apply(params, state.obs)
    pi, _ = net.apply(params, obs)
    action = pi.sample(seed=_rng)
    start_time = time.time()   
    #state = step_fn(state, action)
    obs, state, reward, done, _ = step_fn(__rng, state, action) 
    print(f"Step time: {time.time() - start_time}")
    total_reward += reward
    
    #if state.done:
    #    break
    #else:
    #    rollout.append(state)
    #if done:
    #    break
    #else:
    #    rollout.append(state)    
    if s > 1000:
        break
    else:
        rollout.append(state)
        
print(f"Total reward: {total_reward}")
    

Step: 1
Step time: 0.0030930042266845703
Step: 2
Step time: 0.0030281543731689453
Step: 3
Step time: 0.0031213760375976562
Step: 4
Step time: 0.0030324459075927734
Step: 5
Step time: 0.002891063690185547
Step: 6
Step time: 0.0026743412017822266
Step: 7
Step time: 0.0025217533111572266
Step: 8
Step time: 0.002278566360473633
Step: 9
Step time: 0.0024955272674560547
Step: 10
Step time: 0.0024023056030273438
Step: 11
Step time: 0.002293109893798828
Step: 12
Step time: 0.0025281906127929688
Step: 13
Step time: 0.0024900436401367188
Step: 14
Step time: 0.0027723312377929688
Step: 15
Step time: 0.002867460250854492
Step: 16
Step time: 0.002699136734008789
Step: 17
Step time: 0.002707958221435547
Step: 18
Step time: 0.002709627151489258
Step: 19
Step time: 0.0032579898834228516
Step: 20
Step time: 0.0029449462890625
Step: 21
Step time: 0.002795696258544922
Step: 22
Step time: 0.002480745315551758
Step: 23
Step time: 0.0026366710662841797
Step: 24
Step time: 0.00261688232421875
Step: 25
Step t

In [40]:
a = html.render(env.sys, [state.env_state.env_state.pipeline_state for state in rollout])

AttributeError: 'LogEnvState' object has no attribute 'pipeline_state'

In [33]:
display(HTML(a))

In [9]:
metrics = out["metrics"]

In [22]:
metrics["returned_episode_lengths"][90:110,:,4]

Array([[119, 119, 119, 119, 119, 119, 119, 119, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241],
       [241, 241, 241, 241, 241, 241, 241, 241, 241, 241

In [28]:
metrics["returned_episode_lengths"][68:90,:,5]

Array([[ 94,  94,  94,  94, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127],
       [127, 127, 127, 127, 127, 127, 127, 127, 127, 127

In [29]:
metrics['returned_episode_returns'][68:90,:,5]

Array([[219.94601, 219.94601, 219.94601, 219.94601, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.09293, 270.09293],
       [270.09293, 270.09293, 270.09293, 270.09293, 270.09293, 270.09293,
        270.09293, 270.09293, 270.092

In [30]:
metrics["returned_episode"][68:90,:,5]

Array([[False, False, False, False,  True, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, 

In [18]:
with open('visualization.html', 'w') as file:
    file.write(html_content)

In [49]:
display(HTML("<div style='color:red'>Hello World!</div>"))

In [21]:
env, env_params = BraxGymnaxWrapper(config["ENV_NAME"]), None

rollout = []
rewards = []

rng, _rng = jax.random.split(rng)
env_state = jax.jit(env.reset)(_rng)
done = False
while not done:
    rollout.append(env_state)
    pi, _ = out["runner_state"][0].apply(out["runner_state"][0].params)
    action = pi.sample(seed=_rng)
    rng, _rng = jax.random.split(rng)
    env_state, _, reward, done, _ = env.step(_rng, env_state, action, None)
    rewards.append(reward)
    
a = html.render(env.sys, [s.qp for s in rollout])
HTML(a)

AttributeError: 'TrainState' object has no attribute 'apply'

In [ ]:
def get_env(env_name):
    if env_name == "hopper_uni":
        episode_length = 1000
        
        env = environments_v1.create(env_name, episode_length=episode_length)
    elif env_name == "halfcheetah_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)
        
    elif env_name == "walker2d_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)	
    elif env_name == "ant_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=True)
    elif env_name == "humanoid_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)	
    '''
    elif env_name == "ant_omni":
        episode_length = 250
        max_bd = 30.

        env = environments.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=False)	
    elif env_name == "humanoid_uni":
        episode_length = 1000
        max_bd = 1.

        env = environments.create(env_name, episode_length=episode_length)	
    else:
        ValueError(f"Environment {env_name} not supported.")
    '''
    return env

In [ ]:
config_dict = {
    "no_agents": 32,
    "batch_size": 32 * 1000,
    "mini_batch_size": 32000,
    "no_epochs": 10,
    "learning_rate": 3e-4,
    "discount_rate": 0.99,
    "clip_param": 0.2,
    "vf_coef": 0.5,
    "gae_lambda": 0.95,
    "env_name": "halfcheetah_uni",
}

# Initialize wandb with the configuration dictionary
wandb.init(project="mcpg", name='PPOish', config=config_dict)

env = get_env(config_dict["env_name"])


policy_hidden_layers = [64, 64]
value_hidden_layers = [64, 64]

policy = MLP(
    hidden_layers_size=policy_hidden_layers,
    action_size=env.action_size,
    activation=nn.tanh,
    hidden_init=jax.nn.initializers.orthogonal(scale=jnp.sqrt(2)),
    mean_init=jax.nn.initializers.orthogonal(scale=0.01),
)

value_net = ValueNet(
    hidden_layers_size=value_hidden_layers,
    hidden_init=jax.nn.initializers.orthogonal(scale=jnp.sqrt(2)),
    value_init=jax.nn.initializers.orthogonal(scale=1.),
    activation=nn.tanh,
)

agent = MCPG(Config(**wandb.config), policy, value_net, env)

random_key = jax.random.PRNGKey(0)
train_state_policy, train_state_value = agent.init(random_key)

num_steps = 1000
log_period = 10

metrics_wandb = dict.fromkeys(["mean loss", "mean reward", "mask", "evaluation", 'time'], jnp.array([]))
eval_num = config_dict["no_agents"]
print(f"Number of evaluations per training step: {eval_num}")
start_time = time.time()
for i in range(num_steps // log_period):
    random_key, subkey = jax.random.split(random_key)
    train_state_policy, train_state_value, current_metrics = agent.train(subkey, train_state_policy, train_state_value, log_period, eval=False)
    timelapse = time.time() - start_time
    print(f"Step {(i+1) * log_period}, Time: {timelapse}")
    
    current_metrics["evaluation"] = jnp.arange(log_period*eval_num*(i+1), log_period*eval_num*(i+2), dtype=jnp.int32)
    current_metrics["time"] = jnp.repeat(timelapse, log_period)
    current_metrics["mean loss"] = jnp.repeat(jnp.mean(current_metrics["loss"]), log_period)
    current_metrics["mean reward"] = jnp.repeat(jnp.mean(jnp.sum(current_metrics["reward"], axis=-1)), log_period)
    current_metrics["mask"] = jnp.repeat(jnp.mean(current_metrics["mask"]), log_period)
    '''
    metrics_wandb = jax.tree_util.tree_map(lambda metric, current_metric: jnp.concatenate([metric, current_metric], axis=0), metrics_wandb, current_metrics)
    
    log_metrics = jax.tree_util.tree_map(lambda metric: metric[-1], metrics_wandb)
    
    wandb.log(log_metrics)
    '''
    
    def update_metrics(old_metrics, new_metrics):
        updated_metrics = {}
        for key in old_metrics:
            if key in new_metrics:
                # Check if old metrics for key is empty, and initialize properly if so
                if old_metrics[key].size == 0:
                    updated_metrics[key] = new_metrics[key]
                else:
                    updated_metrics[key] = jnp.concatenate([old_metrics[key], new_metrics[key]], axis=0)
            else:
                raise KeyError(f"Key {key} not found in new metrics.")
        return updated_metrics

    # In your training loop:
    try:
        metrics_wandb = update_metrics(metrics_wandb, current_metrics)
        log_metrics = {k: v[-1] for k, v in metrics_wandb.items()}  # Assuming you want the latest entry
        wandb.log(log_metrics)
    except Exception as e:
        print(f"Error updating metrics: {e}")

    
    
    
    
    start_time = time.time()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: k_mitsides (mitsides). Use `wandb login --relogin` to force relogin


NameError: name 'MLP' is not defined